<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-CIFAR_10/Identification_Objets-ResConv-VariantesArchitectures-CIFAR_10.ipynb" style="float:left;" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Identification d'objets à partir de photos - jeu de données CIFAR-10
## Labo réseau convolutif - variantes d'architecture

Inspirations: 

* TensorFlow Tutorial - Google: <a href="https://www.tensorflow.org/tutorials/images/cnn?hl=fr" target='_blank'>Tutoriel TensorFlow - Réseau neuronal convolutif</a>
* Machine Learning Mastery - Jason Brownlee: <a href="https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/" target='_blank'>How to Develop a CNN From Scratch for CIFAR-10 Photo Classification</a> 

## Importation des bibliothèques utilisées pour le tutoriel


In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:",tf.__version__)
import keras
print("Keras version:",keras.__version__)


## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/><br>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 21

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Jeu de données - photos CIFAR-10
L'ensemble de données CIFAR-10 (Canadian Institute For Advanced Research) comporte 60 000 photographies en couleur de 32×32 pixels d'objets de 10 classes différentes. Il est relativement simple d'atteindre une précision de 80 %. On peut obtenir des performances de 90 % avec ces données avec des réseaux neuronaux convolutifs. 

* 0 : avion
* 1 : automobile
* 2 : oiseau
* 3 : chat
* 4 : cerf
* 5 : chien
* 6 : grenouille
* 7 : cheval
* 8 : bateau
* 9 : camion


In [ ]:
# Importer le jeu de données CIFAR-10
from keras.datasets import cifar10

dic_noms_classe = { 
    0 : "avion",
    1 : "automobile",
    2 : "oiseau",
    3 : "chat",
    4 : "cerf",
    5 : "chien",
    6 : "grenouille",
    7 : "cheval",
    8 : "bateau",
    9 : "camion",
}

# Lire le jeu de données CIFAR-10 et le diviser entre
# les données d'entrainement et les données de test
(attributs_entrainement, classes_cibles_entrainement), (attributs_test, classes_cibles_test) = cifar10.load_data()

# résumé des données 
print()
print('Entraînement: attributs=%s, classes-cibles=%s' % (attributs_entrainement.shape, classes_cibles_entrainement.shape))
print('Test: attributs=%s, classes-cibles=%s' % (attributs_test.shape, classes_cibles_test.shape))

# Afficher les 24 premières images
print()
print("Quelques images avec leur étiquette de classe-cible...")
%matplotlib inline
# définir lagrill d'affichage des images
fig, axes = plt.subplots(nrows=4,ncols=6,figsize=(10,8))
for i_rangee in range(0,4):
    for i_colonne in range(0,6):
        axes[i_rangee,i_colonne].set_title(dic_noms_classe[int(classes_cibles_entrainement[i_rangee*6+i_colonne])],
                                           fontsize=10)
        axes[i_rangee,i_colonne].imshow(attributs_entrainement[i_rangee*6+i_colonne])
plt.show()

## Prétraitement des données

In [ ]:
# Conversion des classes-cibles en vecteurs binaires à un bit discriminant
from tensorflow.keras.utils import to_categorical

classes_cibles_entrainement = to_categorical(classes_cibles_entrainement)
classes_cibles_test = to_categorical(classes_cibles_test)

# Normalisation

def normalisation(entrainement, test):
    # convertir de nombres entiers à nombres décimaux
    entrainement_normalise = entrainement.astype('float32')
    test_normalise = test.astype('float32')
    # normalisation à un nombre entre 0 et 1
    entrainement_normalise = entrainement_normalise / 255.0
    test_normalise = test_normalise / 255.0
    return entrainement_normalise, test_normalise

attributs_entrainement, attributs_test = normalisation(attributs_entrainement, attributs_test)

print("Normalisation des données terminée!")

# 1- Modèle de base à 1 couche convolutive

## Construction du modèle de base

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

print("Création d'un modèle de base...")

input_shape = (32, 32, 3)
batch_size = 32
# ou nombre de classes
nombre_classes_cibles = 10

modele_de_base = Sequential()
# Apprentissage et extraction des attributs
modele_de_base.add(Conv2D(32, 
                          kernel_size=(3,3),
                          activation='relu',
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),
                          input_shape=input_shape))
modele_de_base.add(MaxPooling2D(pool_size=(2,2)))

# Classification des images
# Classificateur perceptron multicouche
modele_de_base.add(Flatten())
modele_de_base.add(Dense(128,
                         activation='relu',
                         kernel_initializer='glorot_uniform'))
modele_de_base.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle de base:")
modele_de_base.summary()

### Compilation du modèle de base

In [ ]:
# Compilation du modèle
print("\nCompilation du modèle de base...")
modele_de_base.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])

## Entraînement du modèle de base

In [ ]:
# Entraînement du modèle
print("\Entraînement du modèle de base...")
batch_size = 128
epochs = 15
traces_entrainement = modele_de_base.fit(attributs_entrainement,
                                         classes_cibles_entrainement,
                                         batch_size=batch_size,
                                         epochs=epochs,
                                         validation_split=0.1)

## Évaluation du modèle de base

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du modèle de base...")
resultats_1 = modele_de_base.evaluate(attributs_test,
                                      classes_cibles_test,
                                      verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_1[1]*100))

## Affichage courbes d'entraînement du modèle de base

In [ ]:
# Affichage des courbes d'entraînement
def afficher_courbes_entraînement(nom_modele,traces_entrainement):
    # Tracer l'erreur
    hauteur = 8
    plt.subplots(figsize=(hauteur,1.618*hauteur))
    plt.subplot(211)
    plt.title("Erreur entropie croisée - " + nom_modele)
    plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
    plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
    plt.ylabel("Erreur")
    plt.xlabel("Nombre d'époques")
    plt.legend()
    plt.show()
    # Tracer l'exactitude
    plt.subplots(figsize=(hauteur,1.618*hauteur))
    plt.subplot(212)
    plt.title("\nExactitude de la classification - " + nom_modele)
    plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
    plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
    plt.ylabel("Exactitude")
    plt.xlabel("Nombre d'époques")
    plt.legend()
    plt.show()

In [ ]:
afficher_courbes_entraînement("Modèle de base",traces_entrainement)

# 2- Modèle à 2 couches convolutives 

## Construction modèle 2 couches

Ici, nous allons écrire une fonction `creer_reseau_convolutif` qui nous permettra de construire des réseaux à couches convolutives de différentes profondeur à partir de 4 paramètres:

<ul>
    <ol>
        <li><b>nbre_couches</b>: un entier pour indiquer le nombre de couches désirées de 1 à 5;</li>
        <li><b>reg_extinction</b>: un booléen (False, True) pour indiquer l'usage de la régularisation par extinction de neurones (<i>dropout</i>);</li>
        <li><b>reg_L2</b>: un booléen (False, True) pour indiquer l'usage de la régularisation par la fonction L2;</li>
        <li><b>normalisation</b>: un booléen (False, True) pour indiquer la normalisation des lots de données.</li>
    </ol>
</ul>

Notez que c'est le genre de travail que la bibliothèque Keras fait, mais à partir d'une IPA (<i>API</i>) de haut niveau, pour construire un réseau avec des instructions TensorFlow de plus bas niveau.  

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from tensorflow.keras import regularizers
    
def creer_reseau_convolutif(nbre_couches=5,reg_extinction=False,reg_L2=False,normalisation=False):
    # Créer le nom du modèle
    nom_modele = creer_nom_modele(nbre_couches,reg_extinction,reg_L2,normalisation)

    # Afficher le nom du modèle 
    print("Création d'un " + nom_modele.replace('_',' ')) 
    # Définir l'entrée des données
    dimensions_entree = (32, 32, 3)
    # nombre de classes-cibles
    nombre_classes_cibles = 10
    # Utilisation de l'API Sequential de Keras
    modele = Sequential()
    # Ajout d'une fonction de régularisation ou régularisateur
    diminution_poids = 0.01
    if (reg_L2):
        regularisateur = regularizers.l2(diminution_poids)
    else:
        regularisateur = None
    # Apprentissage et extraction des attributs
    # Boucler sur le nombre de couches desirées
    nombre_filtres = 16
    for index_couche in range(1,nbre_couches+1):
        # Couche convolutive
        # L'augmentation du nombre de filtres en fonction de la
        # profondeur suit une règle simple. Elle double simplement. 
        nombre_filtres *= 2
        modele.add(Conv2D(filters=nombre_filtres,
                          kernel_size=(3,3),
                          activation='relu',
                          kernel_regularizer = regularisateur,
                          kernel_initializer='glorot_uniform',
                          padding='same',
                          strides=(1,1),
                          input_shape=dimensions_entree))
        # Normalisation des mini-lots 
        if (normalisation):
            modele.add(BatchNormalization())
        # Couche de sous-échantillonnage
        modele.add(MaxPooling2D(pool_size=(2,2)))
        if (reg_extinction):
            # Régularisation par extinction de neurones (dropout) 
            modele.add(Dropout(0.2))
    # Classification
    modele.add(Flatten())
    modele.add(Dense(units=128,
                     activation='relu',
                     kernel_initializer='glorot_uniform'))
    # Sortie fonction exponentielle normalisée ou Softmax
    modele.add(Dense(nombre_classes_cibles,
                     activation='softmax'))
    return modele

def creer_nom_modele(nbre_couches=1,reg_extinction=False,reg_L2=False,normalisation=False):
    nom_modele = "réseau_convolutif_" + str(nbre_couches) + "_couches"
    if (reg_extinction or reg_L2):
        nom_modele += "_régularisé"
    if (reg_extinction):
        nom_modele += "_extinction"
    if (reg_L2):
        nom_modele += "_L2"
    if "(" in nom_modele:
        nom_modele += ")"
    if (normalisation):
        nom_modele += "_et_normalisation_des_lots"    
    return nom_modele

In [ ]:
# Construction du modèle
modele_2_couches = creer_reseau_convolutif(nbre_couches=2,
                                           reg_extinction=False,
                                           reg_L2=False,
                                           normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=2,
                              reg_extinction=False,
                              reg_L2=False,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_2_couches.summary()

### Compilation modèle 2 couches

In [ ]:
# Compilation du modèle
print("\nCompilation d'un " + nom_modele.replace('_',' ')+"...")
modele_2_couches.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])

## Entraînement modèle 2 couches

In [ ]:
# Entraînement du modèle
print("\Entraînement du " + nom_modele.replace('_',' ')+"...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_2_couches = modele_2_couches.fit(attributs_entrainement,
                                                     classes_cibles_entrainement,
                                                     batch_size=taille_lots,
                                                     epochs=nbr_iterations,
                                                     validation_split=0.1)

## Évaluation modèle 2 couches

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_2 = modele_2_couches.evaluate(attributs_test,
                                        classes_cibles_test,
                                        verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_2[1]*100))

## Affichage courbes d'entraînement, modèle 2 couches

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_2_couches)

# 3- Modèle à 3 couches convolutives

## Construction modèle 3 couches 

In [ ]:
# Construction du modèle
modele_3_couches = creer_reseau_convolutif(nbre_couches=3,
                                           reg_extinction=False,
                                           reg_L2=False,
                                           normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=False,
                              reg_L2=False,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches.summary()

### Compilation modèle 3 couches 

In [ ]:
# Compilation du modèle
print("\nCompilation d'un " + nom_modele.replace('_',' ')+"...")
modele_3_couches.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])

## Entraînement modèle 3 couches

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ')+"...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches = modele_3_couches.fit(attributs_entrainement,
                                                     classes_cibles_entrainement,
                                                     batch_size=taille_lots,
                                                     epochs=nbr_iterations,
                                                     validation_split=0.1)

## Évaluation modèle 3 couches

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_3 = modele_3_couches.evaluate(attributs_test,
                                        classes_cibles_test,
                                        verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_3[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches)

# 4- Modèle à 4 couches convolutives

## Construction modèle 4 couches

In [ ]:
# Construction du modèle
modele_4_couches = creer_reseau_convolutif(nbre_couches=4,
                                           reg_extinction=False,
                                           reg_L2=False,
                                           normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=4,
                              reg_extinction=False,
                              reg_L2=False,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_4_couches.summary()

### Compilation modèle 4 couches

In [ ]:
# Compilation du modèle
print("\nCompilation d'un " + nom_modele.replace('_',' ')+"...")
modele_4_couches.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])

## Entraînement modèle 4 couches

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ')+"...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_4_couches = modele_4_couches.fit(attributs_entrainement,
                                                     classes_cibles_entrainement,
                                                     batch_size=taille_lots,
                                                     epochs=nbr_iterations,
                                                     validation_split=0.1)

## Évaluation modèle 4 couches

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_4 = modele_4_couches.evaluate(attributs_test,
                                        classes_cibles_test,
                                        verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_4[1]*100))

## Affichage courbes d'entraînement, modèle 4 couches

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_4_couches)

# 5- Modèle 3 couches régularisé par extinction

## Construction modèle 3 couches régularisé par extinction

In [ ]:
# Construction du modèle
modele_3_couches_regularise_extinction = creer_reseau_convolutif(nbre_couches=3,
                                                                 reg_extinction=True,
                                                                 reg_L2=False,
                                                                 normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=True,
                              reg_L2=False,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction.summary()


### Compilation modèle 3 couches régularisé par extinction

In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction.compile(loss="categorical_crossentropy",
                                               optimizer="adam",
                                               metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé par extinction

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' '))
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_extinction = modele_3_couches_regularise_extinction.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

## Évaluation modèle 3 couches régularisé par extinction

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' '))
resultats_5 = modele_3_couches_regularise_extinction.evaluate(attributs_test,
                                                              classes_cibles_test,
                                                              verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_5[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé par extinction 

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_extinction)

# 6- Modèle 3 couches régularisé L2 


## Construction modèle 3 couches régularisé L2

In [ ]:
# Construction du modèle
modele_3_couches_regularise_L2 = creer_reseau_convolutif(nbre_couches=3,
                                                         reg_extinction=False,
                                                         reg_L2=True,
                                                         normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=False,
                              reg_L2=True,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_L2.summary()

### Compilation modèle 3 couches régularisé L2

In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_L2.compile(loss="categorical_crossentropy",
                                       optimizer="adam",
                                       metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé L2

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ') + "...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_L2 = modele_3_couches_regularise_L2.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

## Évaluation modèle 3 couches régularisé L2

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_6 = modele_3_couches_regularise_L2.evaluate(attributs_test,
                                                      classes_cibles_test,
                                                      verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_6[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé L2 

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_L2)

# 7- Modèle 3 couches régularisé (extinction + L2)


## Construction modèle 3 couches régularisé extinction + L2

In [ ]:
# Construction du modèle
modele_3_couches_regularise_extinction_L2 = creer_reseau_convolutif(nbre_couches=3,
                                                                    reg_extinction=True,
                                                                    reg_L2=True,
                                                                    normalisation=False)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=True,
                              reg_L2=True,
                              normalisation=False)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction_L2.summary()


### Compilation modèle 3 couches régularisé (extinction + L2)

In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction_L2.compile(loss="categorical_crossentropy",
                                                  optimizer="adam",
                                                  metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé (extinction + L2)

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ') + "...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_extinction_L2 = modele_3_couches_regularise_extinction_L2.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

## Évaluation modèle 3 couches régularisé (extinction + L2)

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_7 = modele_3_couches_regularise_extinction_L2.evaluate(attributs_test,
                                                                 classes_cibles_test,
                                                                 verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_7[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé (extinction + L2) 

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_extinction_L2)

# 8- Modèle 3 couches régularisé par extinction et normalisation des lots


## Construction modèle 3 couches régularisé par extinction et normalisation des lots

In [ ]:
# Construction du modèle
modele_3_couches_regularise_extinction_et_normalisation = creer_reseau_convolutif(nbre_couches=3,
                                                                                  reg_extinction=True,
                                                                                  reg_L2=False,
                                                                                  normalisation=True)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=True,
                              reg_L2=False,
                              normalisation=True)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction_et_normalisation.summary()


### Compilation modèle 3 couches régularisé par extinction et normalisation des lots

In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))

modele_3_couches_regularise_extinction_et_normalisation.compile(loss="categorical_crossentropy", 
                                                                optimizer="adam",
                                                                metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé par extinction et normalisation des lots

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ') + "...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_extinction_et_normalisation = modele_3_couches_regularise_extinction_et_normalisation.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

352/352 [==============================] - ETA: 0s - loss: 0.6422 - accuracy: 0.7716

## Évaluation modèle 3 couches régularisé par extinction et normalisation des lots

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_8 = modele_3_couches_regularise_extinction_et_normalisation.evaluate(attributs_test,
                                                                               classes_cibles_test,
                                                                               verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_8[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé par extinction et normalisation des lots

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_extinction_et_normalisation)

# 9- Modèle 3 couches régularisé par L2 et normalisation des lots

## Construction modèle 3 couches régularisé par L2 et normalisation des lots

In [ ]:
# Construction du modèle
modele_3_couches_regularise_L2_et_normalisation = creer_reseau_convolutif(nbre_couches=3,
                                                                          reg_extinction=False,
                                                                          reg_L2=True,
                                                                          normalisation=True)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=False,
                              reg_L2=True,
                              normalisation=True)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_L2_et_normalisation.summary()

### Compilation modèle 3 couches régularisé par L2 et normalisation des lots


In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_L2_et_normalisation.compile(loss="categorical_crossentropy",
                                                        optimizer="adam",
                                                        metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé par L2 et normalisation des lots

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ') + "...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_L2_et_normalisation = modele_3_couches_regularise_L2_et_normalisation.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

## Évaluation modèle 3 couches régularisé par L2 et normalisation des lots


In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_9 = modele_3_couches_regularise_L2_et_normalisation.evaluate(attributs_test,
                                                                        classes_cibles_test,
                                                                        verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_9[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé par L2 et normalisation des lots

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_L2_et_normalisation)

# 10- Modèle 3 couches régularisé (extinction + L2) et normalisation des lots

## Construction modèle 3 couches régularisé (extinction + L2) et normalisation des lots

In [ ]:
# Construction du modèle
modele_3_couches_regularise_extinction_L2_et_normalisation = creer_reseau_convolutif(nbre_couches=3,
                                                                                     reg_extinction=True,
                                                                                     reg_L2=True,
                                                                                     normalisation=True)  
nom_modele = creer_nom_modele(nbre_couches=3,
                              reg_extinction=True,
                              reg_L2=True,
                              normalisation=True)
print("\nDescription du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction_L2_et_normalisation.summary()

### Compilation modèle 3 couches régularisé (extinction + L2) et normalisation des lots

In [ ]:
# Compilation du modèle
print("\nCompilation du " + nom_modele.replace('_',' '))
modele_3_couches_regularise_extinction_L2_et_normalisation.compile(loss="categorical_crossentropy",
                                                                   optimizer="adam",
                                                                   metrics=["accuracy"])

## Entraînement modèle 3 couches régularisé (extinction + L2) et normalisation des lots

In [ ]:
# Entraînement du modèle
print("\nEntraînement du " + nom_modele.replace('_',' ') + "...")
taille_lots = 128
nbr_iterations = 15
traces_entrainement_3_couches_regularise_extinction_L2_et_normalisation = modele_3_couches_regularise_extinction_L2_et_normalisation.fit(
    attributs_entrainement,
    classes_cibles_entrainement,
    batch_size=taille_lots,
    epochs=nbr_iterations,
    validation_split=0.1)

## Évaluation modèle 3 couches régularisé (extinction + L2) et normalisation des lots

In [ ]:
# Évaluation du modèle
print("\nÉvaluation du " + nom_modele.replace('_',' ') + "...")
resultats_10 = modele_3_couches_regularise_extinction_L2_et_normalisation.evaluate(attributs_test,
                                                                                   classes_cibles_test,
                                                                                   verbose=0)
print("Exactitude test: {:.2f}%".format(resultats_10[1]*100))

## Affichage courbes d'entraînement, modèle 3 couches régularisé (extinction + L2) et normalisation des lots

In [ ]:
afficher_courbes_entraînement(nom_modele.replace('_',' '),
                              traces_entrainement_3_couches_regularise_extinction_L2_et_normalisation)

# -----------------------------------------------------------------------------------
# Affichage des résultats comparés des différentes architectures

## Afficher le meilleur résultats et le nom du modèle correspondant


In [ ]:
# Préparation d'une liste regroupant tous les résultats
resultats = [resultats_1[1],
             resultats_2[1],
             resultats_3[1],
             resultats_4[1],
             resultats_5[1],
             resultats_6[1],
             resultats_7[1],
             resultats_8[1],
             resultats_9[1],
             resultats_10[1],
            ]
print(resultats)

# Préparation d'une liste regroupant tous les noms de modèles
noms_modeles = ['1 couche',
                '2 couches', 
                '3 couches', 
                '4 couches',
                '3 couches, regularisé extinction',
                '3 couches, regularisé L2',
                '3 couches, regularisé extinction + L2',
                '3 couches, regularisé extinction, lots normalisés',
                '3 couches, regularisé L2, lots normalisés',
                '3 couches, regularisé extinction + L2, lots normalisés'
               ]

# Afficher le meilleur résultats et le nom du modèle correspondant
import numpy as np
index_meilleur_resultat = np.argmax(np.asarray(resultats))
print("\nMeilleur résultat: {:.2f}%".format(resultats[index_meilleur_resultat]*100))
print("Meilleur modèle: {}".format(noms_modeles[index_meilleur_resultat]))

In [ ]:
# Affichage des résultats comparés des différentes architectures

hauteur = 6
plt.figure(figsize=(hauteur*1.62,hauteur))
plt.title("Résultats comparés des différentes architectures")
plt.plot(noms_modeles, resultats, color='orange')
plt.ylabel("Exactitude")
plt.xlabel("Noms des modèles")
plt.axvline(x=index_meilleur_resultat, color='g', ls=':')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
print("Carnet IPython exécution terminée!")